# test 
Test the connection with a single call


In [ ]:
#imports
import requests
import time

boxGTal = '593fc68947df4f001113f9eb'
baseURLOpenSense ='https://api.opensensemap.org/boxes/'
try:
    resp = requests.get(baseURLOpenSense+boxGTal)
    box = resp.json()
except TimeoutError:
    print('there was an timeout error')
#except:
#    print("Unexpected error:")

In [ ]:
box

# read every interval

In [1]:
#%pip install schedule

In [2]:
from confluent_kafka import Producer

In [3]:
p = Producer({'bootstrap.servers': 'kafka-1:19092,kafka-2:19093'})
boxGTal = '593fc68947df4f001113f9eb'
baseURLOpenSense ='https://api.opensensemap.org/boxes/'
topicName="raw"

In [5]:
def delivery_report(err, msg):
    """ Called once for each message produced to indicate delivery result.
        Triggered by poll() or flush(). """
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))


In [7]:
import schedule
import time
import requests

lastone = ""
def printTemperatureFromBox(s):
    print('{} temperature: {} {}'
              .format(s['lastMeasurement']['createdAt'],
                         s['lastMeasurement']['value'],
                         s['unit']))
        
def job():
    onlyNew= False
    global lastone
    lastRead = lastone
    #print("start reading")
    resp = requests.get(baseURLOpenSense+boxGTal)
    #print("finished reading")
    box = resp.json()
    for s in box['sensors']:   
        if s['title'] == "Temperatur":
            isNewValue= not lastRead or (
                lastRead['lastMeasurement']['createdAt'] != 
                s['lastMeasurement']['createdAt'])
            if(not onlyNew or isNewValue):
                printTemperatureFromBox(s)
                lastone = s                
                p.poll(0)
                # Asynchronously produce a message, the delivery report callback
                # will be triggered from poll() above, or flush() below, when the message has
                # been successfully delivered or failed permanently.
                # p.produce(topicName, data.encode('utf-8'), callback=delivery_report)
                data =  s['lastMeasurement']['value']
                p.produce(topicName, key="593fc68947df4f001113f9eb", value = data.encode('utf-8'), callback=delivery_report)

                # Wait for any outstanding messages to be delivered and delivery report
                # callbacks to be triggered.
                p.flush()

# schedule the job every minute  
schedule.every(10).seconds.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

2020-05-15T08:45:50.391Z temperature: 12.10 °C
Message delivered to raw [3]
2020-05-15T08:45:50.391Z temperature: 12.10 °C
Message delivered to raw [3]
2020-05-15T08:45:50.391Z temperature: 12.10 °C
Message delivered to raw [3]
2020-05-15T08:45:50.391Z temperature: 12.10 °C
Message delivered to raw [3]
2020-05-15T08:45:50.391Z temperature: 12.10 °C
Message delivered to raw [3]


KeyboardInterrupt: 